In [3]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
import csv
import optuna

In [4]:
# 셀에서 출력 정확도 설정.
%precision %.3f

'%.3f'

In [5]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

import random

documents = documents[:77] + random.sample(documents[-142:-15], 62) + documents[-15:]

In [6]:
# 전문가 평점 3점 이상인 상위 175개는 label에 1, 나머지 하위는 label에 0 값 부여
# 분류 예측용 label
labels = []
for i in range(154):
    if i < 77:
        labels.append(1)
    else:
        labels.append(0)

In [7]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(documents_corpus, labels, test_size=0.3, random_state=42)

In [8]:
# Tagged Corpus 생성.
# 모델 학습에 필요한 형태로 변환.
my_tagged_corpus = [TaggedDocument(words=my_words, tags=[i]) for i, my_words in enumerate(X_train)]

In [9]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=42)
    model.train(my_tagged_corpus, total_examples=model.corpus_count, epochs=model.epochs)

    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Randomforest 하이퍼파라미터 튜닝
    n_estimators = trial.suggest_int('n_estimators', 50, 500, step=10)
    max_depth = trial.suggest_int('max_depth', 5, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 5)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])

    # Randomforest 모델 생성
    model = RandomForestClassifier(n_estimators=n_estimators,
                                    max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features,
                                    random_state=42)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-27 16:52:41,501] A new study created in memory with name: no-name-513b2044-a1ca-4bf1-9afb-fd8e92770ee9
[I 2024-06-27 16:52:55,767] Trial 6 finished with value: 0.6222222222222222 and parameters: {'vector_size': 50, 'window': 8, 'min_count': 9, 'epochs': 20, 'negative': 14, 'dm': 0, 'n_estimators': 130, 'max_depth': 22, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 6 with value: 0.6222222222222222.
[I 2024-06-27 16:52:58,397] Trial 0 finished with value: 0.5777777777777777 and parameters: {'vector_size': 50, 'window': 3, 'min_count': 3, 'epochs': 25, 'negative': 12, 'dm': 0, 'n_estimators': 70, 'max_depth': 29, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 6 with value: 0.6222222222222222.
[I 2024-06-27 16:53:02,138] Trial 7 finished with value: 0.7058823529411765 and parameters: {'vector_size': 110, 'window': 3, 'min_count': 3, 'epochs': 30, 'negative': 10, 'dm': 0, 'n_estimators': 390, 'max_dept

Best hyperparameters:  {'vector_size': 230, 'window': 10, 'min_count': 8, 'epochs': 35, 'negative': 10, 'dm': 1, 'n_estimators': 340, 'max_depth': 19, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


In [10]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_rf = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=42)
model_rf.build_vocab(my_tagged_corpus)
model_rf.train(my_tagged_corpus, total_examples=model_rf.corpus_count, epochs=model_rf.epochs)
# RandomForest 모델 생성
clf_rf = RandomForestClassifier(n_estimators=best_params['n_estimators'],
                            max_depth=best_params['max_depth'],
                            min_samples_split=best_params['min_samples_split'],
                            min_samples_leaf=best_params['min_samples_leaf'],
                            max_features=best_params['max_features'],
                            random_state=42)

# Doc2Vec 벡터 생성
X_train_vec = [model_rf.infer_vector(doc) for doc in X_train]
X_test_vec = [model_rf.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_rf.fit(X_train_vec, y_train)
y_pred = clf_rf.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.79      0.76      0.78        25
           1       0.74      0.77      0.76        22

    accuracy                           0.77        47
   macro avg       0.77      0.77      0.77        47
weighted avg       0.77      0.77      0.77        47



In [11]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300, step=10)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])
    
    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=42)
    model.build_vocab(my_tagged_corpus)
    model.train(my_tagged_corpus, total_examples=model.corpus_count, epochs=model.epochs)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # SVM 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 100.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
    gamma = trial.suggest_float('gamma', 1e-4, 1.0, log=True) if kernel in ['rbf', 'poly'] else 'scale'

    # SVM 모델 생성
    model = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        gamma=gamma,
        random_state=42
    )

    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-27 17:10:23,029] A new study created in memory with name: no-name-d6b83861-29a3-4fdc-9f64-5e2903d55f25
[I 2024-06-27 17:10:42,665] Trial 1 finished with value: 0.6792452830188679 and parameters: {'vector_size': 300, 'window': 5, 'min_count': 8, 'epochs': 30, 'negative': 5, 'dm': 0, 'C': 41.00792385034889, 'kernel': 'poly', 'degree': 2, 'gamma': 0.0026460673676160177}. Best is trial 1 with value: 0.6792452830188679.
[I 2024-06-27 17:10:43,579] Trial 5 finished with value: 0.6538461538461539 and parameters: {'vector_size': 150, 'window': 7, 'min_count': 9, 'epochs': 25, 'negative': 15, 'dm': 0, 'C': 0.021439217646988872, 'kernel': 'linear'}. Best is trial 1 with value: 0.6792452830188679.
[I 2024-06-27 17:10:48,717] Trial 3 finished with value: 0.6470588235294118 and parameters: {'vector_size': 260, 'window': 4, 'min_count': 8, 'epochs': 35, 'negative': 13, 'dm': 0, 'C': 0.3918378886974997, 'kernel': 'poly', 'degree': 5, 'gamma': 0.03995692875302955}. Best is trial 1 with valu

Best hyperparameters:  {'vector_size': 200, 'window': 6, 'min_count': 5, 'epochs': 50, 'negative': 10, 'dm': 1, 'C': 15.545256572084925, 'kernel': 'rbf', 'gamma': 0.00565597972157351}


In [12]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_svc = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=42)
model_svc.build_vocab(my_tagged_corpus)
model_svc.train(my_tagged_corpus, total_examples=model_svc.corpus_count, epochs=model_svc.epochs)
# SVM 분류 모델 생성
clf_svc = SVC(C=best_params['C'], kernel=best_params['kernel'], gamma=best_params['gamma'], random_state=42)#degree=best_params['degree'], gamma=best_params['gamma'], 

# Doc2Vec 벡터 생성
X_train_vec = [model_svc.infer_vector(doc) for doc in X_train]
X_test_vec = [model_svc.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_svc.fit(X_train_vec, y_train)
y_pred = clf_svc.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.72      0.84        25
           1       0.76      1.00      0.86        22

    accuracy                           0.85        47
   macro avg       0.88      0.86      0.85        47
weighted avg       0.89      0.85      0.85        47



In [13]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=42)
    model.build_vocab(my_tagged_corpus)
    model.train(my_tagged_corpus, total_examples=model.corpus_count, epochs=model.epochs)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # XGBOOST 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    gamma = trial.suggest_float('gamma', 0, 1)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    
    # XGboost 모델 생성
    model = XGBClassifier(max_depth=max_depth,
                        n_estimators=n_estimators,
                        learning_rate=learning_rate,
                        gamma=gamma,
                        min_child_weight=min_child_weight,
                        random_state=42)
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-27 17:30:51,759] A new study created in memory with name: no-name-f1f20dd5-bf09-40d3-8af2-9a422a3b33f2
[I 2024-06-27 17:31:06,318] Trial 0 finished with value: 0.6376811594202898 and parameters: {'vector_size': 203, 'window': 10, 'min_count': 8, 'epochs': 25, 'negative': 11, 'dm': 0, 'max_depth': 10, 'n_estimators': 128, 'learning_rate': 0.15345689037457078, 'gamma': 0.6886458021154438, 'min_child_weight': 19}. Best is trial 0 with value: 0.6376811594202898.
[I 2024-06-27 17:31:07,542] Trial 3 finished with value: 0.7037037037037037 and parameters: {'vector_size': 87, 'window': 3, 'min_count': 10, 'epochs': 25, 'negative': 15, 'dm': 0, 'max_depth': 11, 'n_estimators': 207, 'learning_rate': 0.21584498888036735, 'gamma': 0.009925846071012434, 'min_child_weight': 5}. Best is trial 3 with value: 0.7037037037037037.
[I 2024-06-27 17:31:10,638] Trial 2 finished with value: 0.45714285714285713 and parameters: {'vector_size': 63, 'window': 8, 'min_count': 7, 'epochs': 35, 'negative'

Best hyperparameters:  {'vector_size': 258, 'window': 5, 'min_count': 10, 'epochs': 20, 'negative': 14, 'dm': 1, 'max_depth': 16, 'n_estimators': 361, 'learning_rate': 0.054256958294788, 'gamma': 0.16215810024223298, 'min_child_weight': 2}


In [17]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_xgb = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=42)
model_xgb.build_vocab(my_tagged_corpus)
model_xgb.train(my_tagged_corpus, total_examples=model_xgb.corpus_count, epochs=model_xgb.epochs)

# XGBoost 모델 생성
clf_xgb = XGBClassifier(
    max_depth=best_params['max_depth'],
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    gamma=best_params['gamma'],
    min_child_weight=best_params['min_child_weight'],
    random_state=42
)

# Doc2Vec 벡터 생성
X_train_vec = [model_xgb.infer_vector(doc) for doc in X_train]
X_test_vec = [model_xgb.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_xgb.fit(X_train_vec, y_train)
y_pred = clf_xgb.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.84      0.79        25
           1       0.79      0.68      0.73        22

    accuracy                           0.77        47
   macro avg       0.77      0.76      0.76        47
weighted avg       0.77      0.77      0.76        47



In [18]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=42)
    model.build_vocab(my_tagged_corpus)
    model.train(my_tagged_corpus, total_examples=model.corpus_count, epochs=model.epochs)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # LightGBM 하이퍼파라미터 튜닝
    max_depth = trial.suggest_int('max_depth', 2, 30)
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 50)

    # LightGBM 모델 생성
    model = LGBMClassifier(
        max_depth=max_depth,
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        min_child_samples=min_child_samples,
        random_state=42
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-27 17:40:29,105] A new study created in memory with name: no-name-035beb88-8ccc-43c1-9085-d20d001f81fd
[I 2024-06-27 17:40:47,369] Trial 3 finished with value: 0.4117647058823529 and parameters: {'vector_size': 212, 'window': 7, 'min_count': 10, 'epochs': 20, 'negative': 8, 'dm': 0, 'max_depth': 2, 'num_leaves': 77, 'learning_rate': 0.06957192888984581, 'n_estimators': 215, 'min_child_samples': 19}. Best is trial 3 with value: 0.4117647058823529.
[I 2024-06-27 17:40:50,036] Trial 5 finished with value: 0.7037037037037037 and parameters: {'vector_size': 123, 'window': 3, 'min_count': 7, 'epochs': 25, 'negative': 6, 'dm': 0, 'max_depth': 11, 'num_leaves': 137, 'learning_rate': 0.22785517655753712, 'n_estimators': 336, 'min_child_samples': 34}. Best is trial 5 with value: 0.7037037037037037.
[I 2024-06-27 17:40:52,733] Trial 4 finished with value: 0.6976744186046512 and parameters: {'vector_size': 191, 'window': 4, 'min_count': 8, 'epochs': 30, 'negative': 3, 'dm': 0, 'max_dept

Best hyperparameters:  {'vector_size': 275, 'window': 2, 'min_count': 9, 'epochs': 30, 'negative': 10, 'dm': 1, 'max_depth': 13, 'num_leaves': 178, 'learning_rate': 0.02338154232402359, 'n_estimators': 83, 'min_child_samples': 46}


In [19]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_lgbm = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=42)
model_lgbm.build_vocab(my_tagged_corpus)
model_lgbm.train(my_tagged_corpus, total_examples=model_lgbm.corpus_count, epochs=model_lgbm.epochs)

# LightGBM 모델 생성
clf_lgbm = LGBMClassifier(max_depth=best_params['max_depth'], num_leaves=best_params['num_leaves'], 
                    learning_rate=best_params['learning_rate'], n_estimators=best_params['n_estimators'], 
                    min_child_samples=best_params['min_child_samples'], random_state=42)


# Doc2Vec 벡터 생성
X_train_vec = [model_lgbm.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lgbm.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lgbm.fit(X_train_vec, y_train)
y_pred = clf_lgbm.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

[LightGBM] [Info] Number of positive: 55, number of negative: 52
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000913 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10254
[LightGBM] [Info] Number of data points in the train set: 107, number of used features: 275
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.514019 -> initscore=0.056089
[LightGBM] [Info] Start training from score 0.056089
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [20]:
# Optuna 목적 함수
def objective(trial):
    # Doc2Vec 하이퍼파라미터 튜닝
    vector_size = trial.suggest_int('vector_size', 50, 300)
    window = trial.suggest_int('window', 2, 10)
    min_count = trial.suggest_int('min_count', 1, 10)
    epochs = trial.suggest_int('epochs', 20, 50, step=5)
    negative = trial.suggest_int('negative', 3, 15)
    dm = trial.suggest_categorical('dm', [0, 1])

    # Doc2Vec 모델 학습
    model = Doc2Vec(my_tagged_corpus, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs, dm=dm, negative=negative, seed=42)
    X_train_vec = [model.infer_vector(doc) for doc in X_train]
    X_test_vec = [model.infer_vector(doc) for doc in X_test]

    # Logistic Regression 하이퍼파라미터 튜닝
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])

    # Logistic Regression 모델 생성
    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=42
    )
    
    model.fit(X_train_vec, y_train)
    score = f1_score(y_test, model.predict(X_test_vec))
    
    return score

# Optuna 최적화 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)

# 최적 하이퍼파라미터 출력
print('Best hyperparameters: ', study.best_params)

[I 2024-06-27 18:11:47,439] A new study created in memory with name: no-name-db97a4de-e259-445d-99b0-86f06ec6b903
[I 2024-06-27 18:11:57,506] Trial 7 finished with value: 0.68 and parameters: {'vector_size': 113, 'window': 10, 'min_count': 7, 'epochs': 25, 'negative': 5, 'dm': 0, 'C': 0.11124529856162127, 'penalty': 'l1', 'solver': 'saga'}. Best is trial 7 with value: 0.68.
[I 2024-06-27 18:11:59,004] Trial 0 finished with value: 0.6376811594202898 and parameters: {'vector_size': 169, 'window': 7, 'min_count': 7, 'epochs': 25, 'negative': 10, 'dm': 0, 'C': 0.03595932318872345, 'penalty': 'l1', 'solver': 'saga'}. Best is trial 7 with value: 0.68.
[I 2024-06-27 18:11:59,114] Trial 4 finished with value: 0.68 and parameters: {'vector_size': 99, 'window': 5, 'min_count': 6, 'epochs': 25, 'negative': 13, 'dm': 0, 'C': 0.06972721499727726, 'penalty': 'l2', 'solver': 'saga'}. Best is trial 7 with value: 0.68.
[I 2024-06-27 18:12:00,095] Trial 3 finished with value: 0.6666666666666666 and para

Best hyperparameters:  {'vector_size': 93, 'window': 4, 'min_count': 7, 'epochs': 50, 'negative': 4, 'dm': 0, 'C': 0.3242574184945189, 'penalty': 'l2', 'solver': 'saga'}


In [21]:
best_params = study.best_params

# Doc2Vec 모델 생성
model_lr = Doc2Vec(my_tagged_corpus, vector_size=best_params['vector_size'], 
                window=best_params['window'], min_count=best_params['min_count'],
                epochs=best_params['epochs'], dm=best_params['dm'], 
                negative=best_params['negative'], seed=42)
model_lr.build_vocab(my_tagged_corpus)
model_lr.train(my_tagged_corpus, total_examples=model_lr.corpus_count, epochs=model_lr.epochs)

# Logistic Regressor 모델 생성
clf_lr = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'], random_state=42)

# Doc2Vec 벡터 생성
X_train_vec = [model_lr.infer_vector(doc) for doc in X_train]
X_test_vec = [model_lr.infer_vector(doc) for doc in X_test]

# 분류 모델 학습 및 예측
clf_lr.fit(X_train_vec, y_train)
y_pred = clf_lr.predict(X_test_vec)

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.68      0.74        25
           1       0.69      0.82      0.75        22

    accuracy                           0.74        47
   macro avg       0.75      0.75      0.74        47
weighted avg       0.75      0.74      0.74        47



c:\Users\user\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [22]:
file_path = r'.\crawling_data\preprocessed_essay.csv'

# 자소서 리스트화
documents = []
with open(file_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        documents.append(row[0])

# 사용하지 않은 2점짜리 자소서 리스트
documents = documents[-175:-77]

In [23]:
# 불용어 리스트 가져오기
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return filtered_tokens

documents_corpus = []
for document in documents:
    documents_corpus.append(preprocess(document))

In [24]:
lr_vec = [model_lr.infer_vector(doc) for doc in documents_corpus]
svc_vec = [model_svc.infer_vector(doc) for doc in documents_corpus]
rf_vec = [model_rf.infer_vector(doc) for doc in documents_corpus]
xgb_vec = [model_xgb.infer_vector(doc) for doc in documents_corpus]
lgbm_vec = [model_lgbm.infer_vector(doc) for doc in documents_corpus]

print(clf_lr.predict(lr_vec).sum())
print(clf_svc.predict(svc_vec).sum())
print(clf_rf.predict(rf_vec).sum())
print(clf_xgb.predict(xgb_vec).sum())
print(clf_lgbm.predict(lgbm_vec).sum())

66
61
45
36
74


In [26]:
import pickle

with open('d2v_model_rf.pkl', 'wb') as f:
    pickle.dump(model_rf, f)

with open('d2v_model_svc.pkl', 'wb') as f:
    pickle.dump(model_svc, f)

with open('d2v_model_xgb.pkl', 'wb') as f:
    pickle.dump(model_xgb, f)

with open('d2v_model_lgbm.pkl', 'wb') as f:
    pickle.dump(model_lgbm, f)

with open('d2v_model_lr.pkl', 'wb') as f:
    pickle.dump(model_lr, f)
    
with open('clf_model_rf.pkl', 'wb') as f:
    pickle.dump(clf_rf, f)

with open('clf_model_svc.pkl', 'wb') as f:
    pickle.dump(clf_svc, f)
    
with open('clf_model_xgb.pkl', 'wb') as f:
    pickle.dump(clf_xgb, f)

with open('clf_model_lgbm.pkl', 'wb') as f:
    pickle.dump(clf_lgbm, f)

with open('clf_model_lr.pkl', 'wb') as f:
    pickle.dump(clf_lr, f)